In [36]:
%load_ext autotime
import numpy as np 
import pandas as pd 
import os
from tqdm import tqdm
tqdm.pandas()
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"  # specify which GPU(s) to be used



The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 4.46 ms


In [37]:
TEXT_COL = 'comment_text'
EMB_PATH = '../input/crawl-300d-2M.vec'
GLOVE_PATH = '../input/glove.840B.300d.txt'
train = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv', index_col='id')
test = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv', index_col='id')

# train['target']= (train['target'].values>0.5).astype(int)


/usr/local/lib/python3.5/dist-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 11.7 s


In [38]:
# print(train['target'].value_counts())

time: 360 µs


In [39]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')

def load_embeddings(embed_dir=EMB_PATH):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in tqdm(open(embed_dir,encoding='utf-8',errors='ignore')))
    return embedding_index

def build_embedding_matrix(word_index, embeddings_index, max_features, lower = True, verbose = True):
    embedding_matrix = np.zeros((max_features, 300))
    for word, i in tqdm(word_index.items(),disable = not verbose):
        if lower:
            word = word.lower()
        if i >= max_features: continue
        try:
            embedding_vector = embeddings_index[word]
        except:
            embedding_vector = embeddings_index["unknown"]
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

def build_matrix(word_index, embeddings_index):
    embedding_matrix = np.zeros((len(word_index) + 1,300))
    for word, i in word_index.items():
        try:
            embedding_matrix[i] = embeddings_index[word]
        except:
            embedding_matrix[i] = embeddings_index["unknown"]
    return embedding_matrix

time: 5.74 ms


In [40]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import gc

maxlen = 220
max_features = 300000
embed_size = 600
tokenizer = Tokenizer(num_words=max_features, lower=True) #filters = ''
#tokenizer = text.Tokenizer(num_words=max_features)
print('fitting tokenizer')
tokenizer.fit_on_texts(list(train[TEXT_COL]) + list(test[TEXT_COL]))
word_index = tokenizer.word_index
X_train = tokenizer.texts_to_sequences(list(train[TEXT_COL]))
y_train = train['target'].values
X_test = tokenizer.texts_to_sequences(list(test[TEXT_COL]))

X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)


# del tokenizer
# gc.collect()




fitting tokenizer
time: 3min 8s


In [41]:
import pickle

with open("lstm_att_toeken.pkl",'wb') as f:
    pickle.dump( tokenizer,f)

time: 578 ms


In [42]:
embeddings_index = load_embeddings()
embeddings_index2 = load_embeddings(GLOVE_PATH)


2000001it [02:25, 13736.73it/s]
2196018it [02:50, 12852.75it/s]

time: 5min 16s


In [43]:
import gc

embedding_matrix1 = build_matrix(word_index, embeddings_index)
del embeddings_index
gc.collect()


0

time: 3.82 s


In [44]:
embedding_matrix2 = build_matrix(word_index, embeddings_index2)
del embeddings_index2
gc.collect()

0

time: 4.73 s


In [45]:
embedding_matrix = np.concatenate([embedding_matrix1,embedding_matrix2],axis=-1)
print(embedding_matrix.shape)

(410047, 600)
time: 2.23 s


In [46]:
del embedding_matrix2
gc.collect()

0

time: 442 ms


In [47]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
import keras
import keras.layers as L
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dense,Input,Flatten,concatenate,Dropout,Lambda
from keras.models import Model
import keras.backend as K
import re
from keras.losses import binary_crossentropy
from keras.callbacks import EarlyStopping, ModelCheckpoint
import codecs
from keras.optimizers import Adam



class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

time: 6.34 ms


In [48]:
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
# Overall
weights = np.ones((len(train),)) / 4
# Subgroup
weights += (train[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) / 4
# Background Positive, Subgroup Negative
weights += (( (train['target'].values>=0.5).astype(bool).astype(np.int) +
   (train[identity_columns].fillna(0).values<0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4
# Background Negative, Subgroup Positive
weights += (( (train['target'].values<0.5).astype(bool).astype(np.int) +
   (train[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4
loss_weight = 1.0 / weights.mean()

y_train = np.vstack([(train['target'].values>=0.5).astype(np.int),weights]).T
print(y_train)
y_aux_train = train[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']].values


[[0.   0.25]
 [0.   0.25]
 [0.   0.25]
 ...
 [0.   0.25]
 [1.   0.5 ]
 [0.   0.25]]
time: 637 ms


In [49]:

adam = Adam(lr=2e-5,decay=0.01)

def custom_loss(y_true, y_pred):
    return binary_crossentropy(K.reshape(y_true[:,0],(-1,1)), y_pred) * y_true[:,1]



def build_model(verbose = False, compile = True):
    sequence_input = L.Input(shape=(maxlen,), dtype='int32')
    embedding_layer = L.Embedding(len(word_index) + 1,
                                600,
                                weights=[embedding_matrix],
                                input_length=maxlen,
                                trainable=False)
    x = embedding_layer(sequence_input)
    x = L.SpatialDropout1D(0.2)(x)
    x = L.Bidirectional(L.CuDNNLSTM(64, return_sequences=True))(x)

    att = Attention(maxlen)(x)
    avg_pool1 = L.GlobalAveragePooling1D()(x)
    max_pool1 = L.GlobalMaxPooling1D()(x)

    x = L.concatenate([att,avg_pool1, max_pool1])

    preds = L.Dense(1, activation='sigmoid')(x)
    
    aux_result = Dense(6, activation='sigmoid',name = 'aux_output')(x)


    model = Model(inputs=[sequence_input], outputs=[preds,aux_result])
#     if verbose:
#         model.summary()
#     if compile:
        
    return model


time: 21.1 ms


In [50]:


# https://www.kaggle.com/hireme/fun-api-keras-f1-metric-cyclical-learning-rate/code

from keras.callbacks import *

class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())
 

time: 6.12 ms


In [52]:

from sklearn.model_selection import StratifiedKFold
import tensorflow  as tf

splits = list(StratifiedKFold(n_splits=10,shuffle=True,random_state=4590).split(X_train,train['target']>0.5))

K.clear_session()
tf.reset_default_graph()



BATCH_SIZE = 2048
NUM_EPOCHS = 100

oof_preds = np.zeros((X_train.shape[0]))
test_preds = np.zeros((X_test.shape[0]))
for fold in  range(10):
    
    print("开始第%d折训练"%(fold+1))
 
    clr = CyclicLR(base_lr=0.001, max_lr=0.002,
                   step_size=300., mode='exp_range',
                   gamma=0.99994)


    
    tr_ind, val_ind = splits[fold]
    ckpt = ModelCheckpoint('lstm_att_%d_fold.hdf5'%(fold+1), save_best_only = True,save_weights_only=True)
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
    model = build_model()
    model.compile(loss=[custom_loss,'binary_crossentropy'], loss_weights=[loss_weight, 1.0], optimizer='adam')
    model.fit([X_train[tr_ind]],
        [y_train[tr_ind],y_aux_train[tr_ind]],
        batch_size=BATCH_SIZE,
        epochs=NUM_EPOCHS,
        validation_data=([X_train[val_ind]], [y_train[val_ind],y_aux_train[val_ind]]),
        callbacks = [clr,es,ckpt])
    print("开始预测")
    oof_preds[val_ind] += model.predict(X_train[val_ind])[0].flatten()
    test_preds += model.predict(X_test)[0].flatten()
    
    K.clear_session()
    tf.reset_default_graph()

test_preds /= 10

开始第1折训练
Train on 1624386 samples, validate on 180488 samples
Epoch 1/100
1624386/1624386 [==============================] - 148s 91us/step - loss: 0.3447 - dense_1_loss: 0.0693 - aux_output_loss: 0.1223 - val_loss: 0.2920 - val_dense_1_loss: 0.0575 - val_aux_output_loss: 0.1074
Epoch 2/100
1624386/1624386 [==============================] - 148s 91us/step - loss: 0.2923 - dense_1_loss: 0.0577 - aux_output_loss: 0.1070 - val_loss: 0.2816 - val_dense_1_loss: 0.0550 - val_aux_output_loss: 0.1050
Epoch 3/100
1624386/1624386 [==============================] - 149s 92us/step - loss: 0.2829 - dense_1_loss: 0.0553 - aux_output_loss: 0.1054 - val_loss: 0.2776 - val_dense_1_loss: 0.0541 - val_aux_output_loss: 0.1039
Epoch 4/100
1624386/1624386 [==============================] - 149s 92us/step - loss: 0.2775 - dense_1_loss: 0.0539 - aux_output_loss: 0.1046 - val_loss: 0.2751 - val_dense_1_loss: 0.0535 - val_aux_output_loss: 0.1034
Epoch 5/100
1624386/1624386 [==============================] - 149s

1624386/1624386 [==============================] - 148s 91us/step - loss: 0.2569 - dense_1_loss: 0.0480 - aux_output_loss: 0.1028 - val_loss: 0.2775 - val_dense_1_loss: 0.0543 - val_aux_output_loss: 0.1031
Epoch 11/100
1624386/1624386 [==============================] - 149s 92us/step - loss: 0.2553 - dense_1_loss: 0.0475 - aux_output_loss: 0.1028 - val_loss: 0.2768 - val_dense_1_loss: 0.0541 - val_aux_output_loss: 0.1031
Epoch 00011: early stopping
开始预测
开始第4折训练
Train on 1624386 samples, validate on 180488 samples
Epoch 1/100
1624386/1624386 [==============================] - 149s 92us/step - loss: 0.3433 - dense_1_loss: 0.0690 - aux_output_loss: 0.1217 - val_loss: 0.2916 - val_dense_1_loss: 0.0575 - val_aux_output_loss: 0.1071
Epoch 2/100
1624386/1624386 [==============================] - 149s 92us/step - loss: 0.2898 - dense_1_loss: 0.0571 - aux_output_loss: 0.1065 - val_loss: 0.2911 - val_dense_1_loss: 0.0577 - val_aux_output_loss: 0.1058
Epoch 3/100
1624386/1624386 [================

1624386/1624386 [==============================] - 150s 92us/step - loss: 0.2549 - dense_1_loss: 0.0473 - aux_output_loss: 0.1030 - val_loss: 0.2815 - val_dense_1_loss: 0.0555 - val_aux_output_loss: 0.1034
Epoch 12/100
1624386/1624386 [==============================] - 150s 92us/step - loss: 0.2526 - dense_1_loss: 0.0467 - aux_output_loss: 0.1029 - val_loss: 0.2798 - val_dense_1_loss: 0.0550 - val_aux_output_loss: 0.1032
Epoch 00012: early stopping
开始预测
开始第7折训练
Train on 1624387 samples, validate on 180487 samples
Epoch 1/100
1624387/1624387 [==============================] - 152s 94us/step - loss: 0.3410 - dense_1_loss: 0.0684 - aux_output_loss: 0.1214 - val_loss: 0.2876 - val_dense_1_loss: 0.0563 - val_aux_output_loss: 0.1069
Epoch 2/100
1624387/1624387 [==============================] - 153s 94us/step - loss: 0.2881 - dense_1_loss: 0.0567 - aux_output_loss: 0.1063 - val_loss: 0.2803 - val_dense_1_loss: 0.0547 - val_aux_output_loss: 0.1049
Epoch 3/100
1624387/1624387 [================

1624388/1624388 [==============================] - 150s 92us/step - loss: 0.2567 - dense_1_loss: 0.0479 - aux_output_loss: 0.1030 - val_loss: 0.2781 - val_dense_1_loss: 0.0544 - val_aux_output_loss: 0.1035
Epoch 11/100
1624388/1624388 [==============================] - 152s 94us/step - loss: 0.2543 - dense_1_loss: 0.0472 - aux_output_loss: 0.1029 - val_loss: 0.2794 - val_dense_1_loss: 0.0548 - val_aux_output_loss: 0.1034
Epoch 00011: early stopping
开始预测
开始第10折训练
Train on 1624388 samples, validate on 180486 samples
Epoch 1/100
1624388/1624388 [==============================] - 150s 93us/step - loss: 0.3430 - dense_1_loss: 0.0687 - aux_output_loss: 0.1227 - val_loss: 0.2920 - val_dense_1_loss: 0.0576 - val_aux_output_loss: 0.1071
Epoch 2/100
1624388/1624388 [==============================] - 150s 92us/step - loss: 0.2889 - dense_1_loss: 0.0569 - aux_output_loss: 0.1062 - val_loss: 0.2840 - val_dense_1_loss: 0.0557 - val_aux_output_loss: 0.1054
Epoch 3/100
1624388/1624388 [===============

In [53]:
oof_preds.shape

(1804874,)

time: 3.97 ms


In [54]:
y_train[:,0].shape

(1804874,)

time: 3.09 ms


In [55]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_train[:,0]>0.5,oof_preds)

0.9619367546469174

time: 783 ms


In [ ]:
submission = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv', index_col='id')
submission['prediction'] = test_preds
submission.reset_index(drop=False, inplace=True)
submission.head()
#%%



In [ ]:
submission.to_csv('submission.csv', index=False)